In [33]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml(name = 'Fashion-MNIST', version=1, return_X_y=True, as_frame=False)
data = fetch_openml(name = 'Fashion-MNIST', version=1, as_frame=False)


/home/workplace/miniconda3/envs/EXAM_DL2/lib/python3.8/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [34]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

In [35]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [36]:
y_int = y.astype(np.uint8)

In [37]:
X_scaled = X / 255

In [38]:
X_train = X_scaled[:60000]
y_train = y_int[:60000]
X_test = X_scaled[60000:]
y_test = y_int[60000:]


In [39]:
class CustomDataset(TensorDataset):
    def __init__(self, X, y, transform=None):
        self.X = torch.tensor(X, dtype=torch.float32, device=DEVICE)
        self.y = torch.tensor(y, dtype=torch.long, device=DEVICE)

    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [40]:
data = CustomDataset(X_scaled, y_int)

trainsize = 0.7
valsize = 0.1
testsize = 0.2

seed = torch.Generator().manual_seed(43)
train_set, val_set, test_set = random_split(data, [trainsize, valsize, testsize], generator=seed)

train_set.__len__(), val_set.__len__(), test_set.__len__()

(49000, 7000, 14000)

In [41]:
BATCHSIZE = 10
trainDL = DataLoader(train_set, batch_size=BATCHSIZE, shuffle=False)
valDL = DataLoader(val_set, batch_size=BATCHSIZE, shuffle=False)
testDL = DataLoader(test_set, batch_size=BATCHSIZE, shuffle=False)

In [46]:
class Model(nn.Module):
    def __init__(self, input, output):
        super().__init__()
        self.seq = nn.Sequential()
        self.seq.add_module('fc1', nn.Linear(input, 100, device=DEVICE))
        self.seq.add_module('relu1', nn.ReLU())
        # self.seq.add_module('fc2', nn.Linear(4000, 100, device=DEVICE))
        # self.seq.add_module('relu2', nn.ReLU())
        self.seq.add_module('fc3', nn.Linear(100, output, device=DEVICE))

    def forward(self, x):
        x = self.seq(x)
        return x

In [60]:
EPOCH = 100
fashion = Model(input=28*28, output=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fashion.parameters(), lr=0.1)

In [61]:
def train(model, train_loader, optimizer, criterion):
    model.train()
    tmploss = []
    for X, y in train_loader:
        y_pred = model(X)
        loss = criterion(y_pred, F.one_hot(y, num_classes=10).float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        tmploss.append(loss.item())
    
    return np.mean(tmploss)

def evaluate(model, val_loader, criterion):
    model.eval()
    tmploss = []
    with torch.no_grad():
        for X, y in val_loader:
            y_pred = model(X)
            loss = criterion(y_pred, F.one_hot(y, num_classes=10).float())
            tmploss.append(loss.item())
    return np.mean(tmploss)

In [62]:


for epoch in range(EPOCH):
    loss_train = train(fashion, trainDL, optimizer, criterion)
    loss_test =  evaluate(fashion, valDL, criterion)
    print(f"Epoch: {epoch}, Train Loss: {loss_train:.4f}, Val Loss: {loss_test:.4f}")

Epoch: 0, Train Loss: 2.3629, Val Loss: 2.3213
Epoch: 1, Train Loss: 2.3250, Val Loss: 2.3213
Epoch: 2, Train Loss: 2.3250, Val Loss: 2.3213
Epoch: 3, Train Loss: 2.3250, Val Loss: 2.3213


KeyboardInterrupt: 